In [1]:
import pandas as pd
import numpy as np
import sympy
from sympy.solvers.inequalities import reduce_rational_inequalities,reduce_inequalities
from itertools import chain
from collections import defaultdict
from IPython.display import display

from sympy.parsing.sympy_parser import parse_expr
from sympy.parsing.sympy_parser import standard_transformations
from sympy.printing.mathml import print_mathml
import math
from IPython.display import display, Markdown, Latex
import os
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

def make_symbols_positive(sym):
    return sympy.symbols("{}".format(sym),positive =True)

def make_symbols_function(sym):
    return sympy.symbols("{}".format(sym),cl)

def subs_function(target,list_):
    return sympy.simplify(target.subs(list_))

def root_cal(value):
    return value**(1/2)

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

print(color.BOLD + 'Hello World !' + color.END)

Hello World !


# w_range

In [3]:
def sol_w_same(df_1,target_):
    w_sol = sympy.solve([df_1["QD"][target_]-df_1["FB"][target_]],w)
    return w_sol

In [4]:
def sol_w_other(df_1,equa):
    w_sol = sympy.solve(equa,w)
    return w_sol

In [5]:
def display_solve(df_1,w_sol):
    for row,col in df_1.iterrows():
        print(row)
        try:
            display(col["QD"].subs([(w,w_sol)]).factor())
        except:
            print()
        print()

In [6]:
def display_solve2(df_1,w_sol):
    for row,col in df_1.iterrows():
        print(row)
        try:
            display(col["QD"].subs([(w,w_sol)]).simplify())
        except:
            print()
        print()

In [7]:
def display_w_insert(df_,w_1):
    order_ = ["w","epsilon","t","p","q_nd","q_t","q","profit_r","profit_s","profit_all","h1_r_p","h1_s_w","h1_s_e"]
    if w_1 == "WH":
        for ord_ in order_:
            try:
                value_ = df_[1][ord_].factor()
                value_2 = df_[1][ord_].simplify()
                print(color.RED + ord_ + color.END)
                display("factor: ",value_)
                display("simplify: ",value_2)
                print("-"*40)
            except:
                print(color.RED + ord_ + color.END)
                print("-"*40)
    else:
        for ord_ in order_:
            try:
                value_ = df_[1][ord_].subs([(w,w_1)]).factor()
                value_2 = df_[1][ord_].subs([(w,w_1)]).simplify()
                print(color.RED + ord_ + color.END)
                display("factor: ",value_)
                display("simplify: ",value_2)
                print("-"*40)
            except:
                print(color.RED + ord_ + color.END)
                print("-"*40)

In [8]:
Path_ = r"C:\Users\Samsung\jupyter\Graduation_thesis\result\version_un_4"

In [9]:
file_list = os.listdir(Path_)
file_list_xlsx = [f for f in file_list if f.endswith(".xlsx")]
#file_FB = [f for f in file_list_xlsx if f.contains("FB")]
#file_WH = [f for f in file_list_xlsx if f.contains("WH")]
#file_QD = [f for f in file_list_xlsx if f.contains("QD")]

In [10]:
file_name = [li.replace(".xlsx","") for li in file_list_xlsx]
file_name

['ND_FB',
 'ND_QD',
 'ND_WH',
 'UN_BD_QD',
 'UN_BD_WH',
 'UN_BS_QD',
 'UN_BS_WH',
 'UN_DD_QD',
 'UN_DD_WH',
 'UN_SD_QD',
 'UN_SD_WH']

In [11]:
for fn in file_name:
    path_ = Path_ + "\\{}.xlsx"
    globals() ["{}".format(fn)] = pd.read_excel(path_.format(fn),index_col=0,header=None)
    try:
        globals() ["{}".format(fn)] = globals() ["{}".format(fn)]#.rename(index={"profit_all":"pro_all"})
    except:
        pass

In [12]:
p,w,c,s,alpha,beta,delta_d,delta_s,epsilon,psi = make_symbols_positive("p,w,c,s,alpha,beta,delta_d,delta_s,epsilon,psi")

In [13]:
parse_dic = {"p":p,"w":w,"c":c,"s":s
             ,"alpha":alpha,"beta":beta
             ,"delta_d":delta_d,"delta_s":delta_s,"epsilon":epsilon }

In [14]:
for fn in file_name:
    globals() ["{}".format(fn)][1] = globals() ["{}".format(fn)][1].map(lambda x: parse_expr(x.replace("^","**").replace("lambda","lambda_"),parse_dic))

# Un cases

In [15]:
solve_ws = sympy.solve((ND_QD[1]["profit_s"]-ND_WH[1]["profit_s"]).factor(),w)
w_w = solve_ws[1]
w_w

(-alpha + 2*beta*c + sqrt(2)*(alpha - beta*c))/beta

In [16]:
solve_w = sympy.solve((ND_QD[1]["profit_all"]-ND_FB[1]["profit_all"]).factor(),w)
w_c = solve_w[0]
w_c

alpha/beta

In [17]:

tar_wh = ND_WH.copy()
tar_qd = ND_QD.copy()

ND_QDW = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_w)]).factor())) 
ND_QDC = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_c)]).factor())) 


In [18]:
time_ = "UN"
dis_ = "DD"


tar_wh = globals() ["{}_{}_WH".format(time_,dis_)]
tar_qd = globals() ["{}_{}_QD".format(time_,dis_)]

globals() ["{}_{}_QD{}".format(time_,dis_,"W")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_w)]).factor()))  
globals() ["{}_{}_QD{}".format(time_,dis_,"C")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_c)]).factor()))  
time_ = "UN"
diss_ = ["DD","SD","BD","BS"]

for dis_ in diss_:
    tar_wh = globals() ["{}_{}_WH".format(time_,dis_)]
    tar_qd = globals() ["{}_{}_QD".format(time_,dis_)]
    globals() ["{}_{}_QD{}".format(time_,dis_,"W")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_w)]).factor()))  
    globals() ["{}_{}_QD{}".format(time_,dis_,"C")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_c)]).factor()))  

In [19]:
time_ = "UN"
dis_ = "SD"


tar_wh = globals() ["{}_{}_WH".format(time_,dis_)]
tar_qd = globals() ["{}_{}_QD".format(time_,dis_)]

globals() ["{}_{}_QD{}".format(time_,dis_,"W")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_w)]).factor()))  
globals() ["{}_{}_QD{}".format(time_,dis_,"C")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_c)]).factor()))  
time_ = "UN"
diss_ = ["DD","SD","BD","BS"]

for dis_ in diss_:
    tar_wh = globals() ["{}_{}_WH".format(time_,dis_)]
    tar_qd = globals() ["{}_{}_QD".format(time_,dis_)]
    globals() ["{}_{}_QD{}".format(time_,dis_,"W")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_w)]).factor()))  
    globals() ["{}_{}_QD{}".format(time_,dis_,"C")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_c)]).factor()))  


In [20]:
time_ = "UN"
dis_ = "BD"


tar_wh = globals() ["{}_{}_WH".format(time_,dis_)]
tar_qd = globals() ["{}_{}_QD".format(time_,dis_)]

globals() ["{}_{}_QD{}".format(time_,dis_,"W")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_w)]).factor()))  
globals() ["{}_{}_QD{}".format(time_,dis_,"C")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_c)]).factor()))  
time_ = "UN"
diss_ = ["DD","SD","BD","BS"]

for dis_ in diss_:
    tar_wh = globals() ["{}_{}_WH".format(time_,dis_)]
    tar_qd = globals() ["{}_{}_QD".format(time_,dis_)]
    globals() ["{}_{}_QD{}".format(time_,dis_,"W")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_w)]).factor()))  
    globals() ["{}_{}_QD{}".format(time_,dis_,"C")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_c)]).factor()))  


In [21]:
time_ = "UN"
dis_ = "BS"


tar_wh = globals() ["{}_{}_WH".format(time_,dis_)]
tar_qd = globals() ["{}_{}_QD".format(time_,dis_)]

globals() ["{}_{}_QD{}".format(time_,dis_,"W")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_w)]).factor()))  
globals() ["{}_{}_QD{}".format(time_,dis_,"C")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_c)]).factor()))  
time_ = "UN"
diss_ = ["DD","SD","BD","BS"]

for dis_ in diss_:
    tar_wh = globals() ["{}_{}_WH".format(time_,dis_)]
    tar_qd = globals() ["{}_{}_QD".format(time_,dis_)]
    globals() ["{}_{}_QD{}".format(time_,dis_,"W")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_w)]).factor()))  
    globals() ["{}_{}_QD{}".format(time_,dis_,"C")] = pd.DataFrame(tar_qd[1].map(lambda x : x.subs([(w,w_c)]).factor()))  


In [22]:
def hessian_(df_,para_):
    try:
        try:
            a_ = float(df_[1]["h1_r_p"].subs(para_))
            b_ = float(df_[1]["h1_s_w"].subs(para_))
            if a_ and b_ <= 0:
                return "NONE"
            else:
                return "STOP"
        except:
            a_ = float(df_[1]["h1_r_p"].subs(para_))
            b_ = float(df_[1]["h1_s_w"].subs(para_))
            c_ = float(df_[1]["h1_s_e"].subs(para_))
            if a_ and b_ and c_ <= 0:
                return "NONE"
            else:
                return "STOP"
    except:
        return "NONE"

In [23]:
def subs_lambda(y,params):
    try:
        out_ = y.subs(params)
        try:
            out_ = round(float(out_),3)
        except:
            pass
    except:
        out_ = 0
    return out_

In [24]:
def apply_num(df,para_list):
    return df.apply(lambda x : x.map(lambda y: subs_lambda(y,para_list)))

In [25]:
file_name_2 = ['ND_FB',
 'ND_QDC',
 'ND_QDW',
 'ND_WH',
 'UN_BD_QDC',
 'UN_BD_QDW',
 'UN_BD_WH',
 'UN_BS_QDC',
 'UN_BS_QDW',
 'UN_BS_WH',
 'UN_DD_QDC',
 'UN_DD_QDW',
 'UN_DD_WH',
 'UN_SD_QDC',
 'UN_SD_QDW',
 'UN_SD_WH']

In [26]:
UN_BD_QDC

,1
0,
p,(alpha + beta*c)/(2*beta)
w,alpha/beta
epsilon,1/beta
q_nd,-(-alpha + beta*c)/2
qd,(-alpha + beta*c)*(delta_d - 1)/2
qs,(-alpha + beta*c)*(delta_s - 1)/2
t_nd,(alpha + beta*c)/(2*beta)
profit_r,(-alpha + beta*c)*(delta_d - delta_s)*(alpha +...
profit_s,-(-alpha + beta*c)**2*(delta_s - 1)/(4*beta)


In [27]:
para_ = [(alpha,1200),(beta,7),(c,50),(delta_d,n_d),(delta_s,n_s),(s,s_)]

NameError: name 'n_d' is not defined

In [28]:

def tar_para(time_,dis_,tar_,n_d,n_s,s_):
    para_ = [(alpha,1200),(beta,7),(c,50),(delta_d,n_d),(delta_s,n_s),(s,s_)]

    tar_wh = globals() ["{}_{}_WH".format(time_,dis_)]
    tar_qdc = globals() ["{}_{}_QDC".format(time_,dis_)]
    tar_qdw = globals() ["{}_{}_QDW".format(time_,dis_)]

    # hessian
    wh_hess = hessian_(tar_wh,para_)
    qdw_hess = hessian_(tar_qdw,para_)
    qdc_hess =hessian_(tar_qdc,para_)
    
    

    wh_df = pd.DataFrame(tar_wh[1].map(lambda x : x.subs(para_).simplify()))  
    qdw_df = pd.DataFrame(tar_qdw[1].map(lambda x : x.subs(para_).simplify()))  
    qdc_df = pd.DataFrame(tar_qdc[1].map(lambda x : x.subs(para_).simplify()))

    p_wh,p_qdw,p_qdc  = float(wh_df[1]["p"]),float(qdw_df[1]["p"]),float(qdc_df[1]["p"])
    
    wh_s = [s_,p_wh,wh_hess,float(wh_df[1][tar_])]
    qdw_s = [s_,p_qdw,qdw_hess,float(qdw_df[1][tar_])]    
    qdc_s = [s_,p_qdc,qdc_hess,float(qdc_df[1][tar_])]   
    return pd.DataFrame([wh_s,qdw_s,qdc_s]).rename(columns={0:"s",1:"price",2:"hessian",3:tar_},index={0:"WH",1:"QDW",2:"QDC"})

In [29]:
tar_para("UN","SD","profit_r",0.3,0.3,111)

,s,price,hessian,profit_r
WH,111,141.071429,NONE,6432.678571
QDW,111,128.497088,NONE,9059.201967
QDC,111,110.714286,NONE,-36.428571


In [ ]:
tar_para("UN","SD","profit_r",0.3,0.3,111)["profit_r"].idxmax()

In [ ]:
## SD 니까 s > price
all_ = []

for delta_1 in np.arange(0,1.1,0.1):
    s_ = 0
    Para_ = [(alpha,1200),(beta,7),(c,50),(delta_d,delta_1),(delta_s,delta_1),(s,s_)]
    tar_1 = "profit_r"
    Time_ = "UN"
    Type_ = "SD"

    Price_ = float(globals () ["{}_{}_WH".format(Time_,Type_)][1]["p"].subs(Para_))

    gap_ = 5
    iter_1 = np.arange(Price_,300,gap_)
    tq = tqdm(iter_1)



    for s_i in tq:
        df_test = tar_para(Time_,Type_,tar_1,delta_1,delta_1,s_i)
        if set(df_test["hessian"].values)=={"NONE"}:
            cand_ = {"Domi":df_test[tar_1].idxmax(),"s":s_i,"delta":delta_1}
            all_.append(cand_)
        else:
            print("None")
    #print(set(df_test["hessian"].values)=={"NONE"})

In [ ]:
pd.DataFrame(all_).to_excel(r"C:\Users\Samsung\jupyter\Graduation_thesis\result\graph_\sense\sd_pir_s_delta.xlsx")

In [121]:
## SD 니까 s > price
all_ = []

for delta_1 in np.arange(0,1.1,0.1):
    s_ = 0
    Para_ = [(alpha,1200),(beta,7),(c,50),(delta_d,delta_1),(delta_s,delta_1),(s,s_)]
    tar_1 = "profit_all"
    Time_ = "UN"
    Type_ = "SD"

    Price_ = float(globals () ["{}_{}_WH".format(Time_,Type_)][1]["p"].subs(Para_))

    gap_ = 5
    iter_1 = np.arange(Price_,300,gap_)
    tq = tqdm(iter_1)



    for s_i in tq:
        df_test = tar_para(Time_,Type_,tar_1,delta_1,delta_1,s_i)
        if set(df_test["hessian"].values)=={"NONE"}:
            cand_ = {"Domi":df_test[tar_1].idxmax(),"s":s_i,"delta":delta_1}
            all_.append(cand_)
        else:
            print("None")
    #print(set(df_test["hessian"].values)=={"NONE"})

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [122]:
pd.DataFrame(all_).to_excel(r"C:\Users\Samsung\jupyter\Graduation_thesis\result\graph_\sense\sd_piall_s_delta.xlsx")

In [111]:
delta_1 = 0.3
s_ = 0
Para_ = [(alpha,1200),(beta,7),(c,50),(delta_d,delta_1),(delta_s,delta_1),(s,s_)]
tar_1 = "profit_s"
Time_ = "UN"
Type_ = "SD"

Price_ = float(globals () ["{}_{}_WH".format(Time_,Type_)][1]["p"].subs(Para_))
Price_

141.07142857142858

# single side 

In [32]:
gap_ = 0.5

iter_1 = np.arange(0,50,gap_)
iter_2 = np.arange(50,s_para(0.3,0.6)[0],gap_)
iter_3 = np.arange(s_para(0.3,0.6)[0],s_para(0.3,0.6)[1],gap_)
iter_4 = np.arange(s_para(0.3,0.6)[1],s_para(0.3,0.6)[1]+42,gap_)

NameError: name 's_para' is not defined

In [140]:
equ_ = (UN_SD_QDW[1]["profit_r"]-UN_SD_WH[1]["profit_r"]).factor()
equ_

(-alpha + beta*c)*(-4*sqrt(2)*alpha + 5*alpha - 4*sqrt(2)*beta*c*delta_s + 4*beta*c*delta_s - 5*beta*c + 4*sqrt(2)*beta*c - 4*beta*delta_s*s + 4*sqrt(2)*beta*delta_s*s)/(16*beta)

In [146]:
sympy.solve(equ_,s)[0].factor()

(-5*alpha + 4*sqrt(2)*alpha - 4*beta*c*delta_s + 4*sqrt(2)*beta*c*delta_s - 4*sqrt(2)*beta*c + 5*beta*c)/(4*beta*delta_s*(-1 + sqrt(2)))

In [136]:
sympy.solve(UN_SD_QDW[1]["profit_r"]-UN_SD_WH[1]["profit_r"],[s,delta_s])[0][0].factor()

(-5*alpha + 4*sqrt(2)*alpha - 4*beta*c*delta_s + 4*sqrt(2)*beta*c*delta_s - 4*sqrt(2)*beta*c + 5*beta*c)/(4*beta*delta_s*(-1 + sqrt(2)))

In [127]:
UN_SD_WH[1]["profit_r"]

(alpha - beta*c)*(2*alpha*delta_s + alpha + 2*beta*c*delta_s - beta*c - 4*beta*delta_s*s)/(16*beta)